In [ ]:
# --- Common imports and paths (add this as Cell 1 in every notebook) ---
import os
import pandas as pd
import numpy as np
import torch
import joblib

# Paths (same as your main project)
PROCESSED_CSV = "./data/processed/processed_sample_200k.csv"
MODEL_PTH = "./models/torch_mlp.pth"
PREPROCESSOR_JOBLIB = "./models/preprocessor.joblib"
METRICS_JOBLIB = "./models/supervised_metrics.joblib"
DISAGREE_CSV = "./analysis/policy_disagreements_sample.csv"


In [23]:
import pandas as pd
df = pd.read_csv("./analysis/disagreements_sample.csv")
print("rows:", len(df))
display(df.head(10))


rows: 200


,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,revol_bal,revol_util,total_acc,open_acc,term_ 36 months,...,addr_state_WV,addr_state_WY,loan_amnt_orig,int_rate_orig,target,pred_prob,pred_label_0.5,action_bestF1,action_05,action_03
0,1.715488,1.716741,2.350939,-1.302153,-0.373509,-0.300163,-1.523684,-0.119975,-0.168278,0.0,...,0.0,0.0,30000.0,19.99,0,0.276534,0,1,0,0
1,2.297386,1.022712,1.399247,-0.726972,-0.373509,0.260680,0.025485,0.459086,-0.700630,0.0,...,0.0,0.0,35000.0,16.99,1,0.231997,0,1,0,0
2,2.297386,1.450697,1.249695,0.467016,-0.373509,1.732231,1.042911,-0.368144,0.541524,0.0,...,0.0,0.0,35000.0,18.84,0,0.279096,0,1,0,0
3,1.133590,0.920922,0.028811,0.221330,3.967157,0.403898,0.766573,-0.368144,-0.345729,0.0,...,0.0,0.0,25000.0,16.55,0,0.224840,0,1,0,0
4,0.240376,0.481370,-0.459271,1.642635,-0.373509,0.189962,-0.355527,0.045471,0.364073,0.0,...,0.0,0.0,17325.0,14.65,0,0.247623,0,1,0,0
5,1.715488,1.314204,0.216430,0.633486,-0.373509,0.507788,1.645831,-0.616313,-0.700630,0.0,...,0.0,0.0,30000.0,18.25,0,0.257614,0,1,0,0
6,-0.612104,0.328684,-0.476946,-0.162124,-0.373509,-0.379630,1.197828,-0.947205,-1.055531,0.0,...,0.0,0.0,10000.0,13.99,0,0.201973,0,1,0,0
7,-0.911782,1.156891,-0.232225,-0.390589,0.711658,-0.607337,-0.740726,1.120871,-0.345729,1.0,...,0.0,0.0,7425.0,17.57,1,0.242579,0,1,0,0
8,1.133590,0.328684,0.610702,-0.234452,-0.373509,-0.091776,-0.874708,0.459086,0.718974,0.0,...,0.0,0.0,25000.0,13.99,0,0.199949,0,1,0,0
9,-0.379345,0.407340,-0.164247,1.821734,0.711658,0.477371,1.160146,0.293640,0.718974,0.0,...,0.0,0.0,12000.0,14.33,1,0.229489,0,1,0,0


In [24]:
# assume columns are 'pred_prob','supervised_action','rl_action' or 'action_bestF1' etc.
# adjust column names to what's in your CSV
sa = 'supervised_action' if 'supervised_action' in df.columns else 'action_bestF1'
ra = 'rl_action' if 'rl_action' in df.columns else 'action_03'  # or whichever you used

counts = df.groupby([sa, ra]).size().unstack(fill_value=0)
print("Counts (supervised_action rows vs rl_action cols):")
print(counts)
# percent where supervised approves but RL denies, and vice versa
total = len(df)
sa_approve_rl_deny = counts.loc[1,0] if (1 in counts.index and 0 in counts.columns) else 0
sa_deny_rl_approve = counts.loc[0,1] if (0 in counts.index and 1 in counts.columns) else 0
print("supervised approves & RL denies:", sa_approve_rl_deny, f"({sa_approve_rl_deny/total:.2%})")
print("supervised denies & RL approves:", sa_deny_rl_approve, f"({sa_deny_rl_approve/total:.2%})")


Counts (supervised_action rows vs rl_action cols):
action_03        0
action_bestF1     
1              200
supervised approves & RL denies: 200 (100.00%)
supervised denies & RL approves: 0 (0.00%)


In [25]:
# sort by predicted probability (highest disagreement) or by loan_amnt_orig
if 'pred_prob' in df.columns:
    sample = df.sort_values('pred_prob', ascending=False).head(20)
elif 'loan_amnt_orig' in df.columns:
    sample = df.sort_values('loan_amnt_orig', ascending=False).head(20)
else:
    sample = df.sample(20, random_state=42)
sample.to_csv("./analysis/disagreements_inspect_top20.csv", index=False)
print("Saved top 20 disagreements to ./analysis/disagreements_inspect_top20.csv")


Saved top 20 disagreements to ./analysis/disagreements_inspect_top20.csv


In [26]:
import numpy as np
proc = pd.read_csv("./analysis/disagreements_sample.csv")  # or read the full predictions file for global calc
# ensure loan_amnt_orig & int_rate_orig exist
loan = pd.to_numeric(proc.get('loan_amnt_orig', pd.Series(1)), errors='coerce').fillna(0).values
ir = pd.to_numeric(proc.get('int_rate_orig', pd.Series(1)), errors='coerce').fillna(0).values / 100.0
target = proc['target'].values

# compute reward function
def reward_for_row(action, idx):
    if action==0:
        return 0.0
    # approve
    return loan[idx]*ir[idx] if target[idx]==0 else -loan[idx]

# vectorized
sa_col = sa
ra_col = ra
idxs = np.arange(len(proc))
sa_vals = proc[sa_col].values.astype(int)
ra_vals = proc[ra_col].values.astype(int)
# rewards
r_sa = np.array([reward_for_row(a,i) for i,a in enumerate(sa_vals)])
r_ra = np.array([reward_for_row(a,i) for i,a in enumerate(ra_vals)])
# mean reward when disagree
dis_mask = (sa_vals != ra_vals)
print("Disagreements rows:", dis_mask.sum())
print("Mean reward - supervised on disagreements:", r_sa[dis_mask].mean())
print("Mean reward - RL/baseline on disagreements:", r_ra[dis_mask].mean())
print("Mean reward delta (RL - supervised):", r_ra[dis_mask].mean() - r_sa[dis_mask].mean())


Disagreements rows: 200
Mean reward - supervised on disagreements: -1404.2685125
Mean reward - RL/baseline on disagreements: 0.0
Mean reward delta (RL - supervised): 1404.2685125


In [28]:
# Robust disagreement inspection (run this cell)
import pandas as pd
import numpy as np
import os

DISAGREE_PATH = "./analysis/disagreements_sample.csv"
PRED_PATH = "./analysis/supervised_predictions.csv"  # full predictions file

# 1) Load file and show columns
df = pd.read_csv(DISAGREE_PATH)
print("Loaded disagreements file rows:", len(df))
print("Columns in disagreements CSV:\n", df.columns.tolist())

# 2) Find supervised-policy column (try several common names)
supervised_candidates = ['supervised_action', 'action_bestF1', 'pred_label_0.5', 'action_05', 'action_03']
supervised_col = next((c for c in supervised_candidates if c in df.columns), None)
if supervised_col is None:
    # try to infer from presence of 'pred_prob' and make supervised_action using best-F1 threshold
    print("No supervised action column found in disagreements CSV. Will try to compute from full predictions file.")
    if os.path.exists(PRED_PATH):
        preds = pd.read_csv(PRED_PATH)
        probs = preds['pred_prob'].values
        y = preds['target'].values
        # compute best-F1 threshold
        from sklearn.metrics import f1_score
        thresholds = np.linspace(0,1,101)
        f1s = [f1_score(y, (probs>=t).astype(int), zero_division=0) for t in thresholds]
        best_t = thresholds[np.argmax(f1s)]
        preds['action_bestF1'] = (probs >= best_t).astype(int)
        supervised_col = 'action_bestF1'
        # merge into df by index if index preserved; else just reassign df to be preds subset based on disagreement indices
        # (simple fallback: save preds file and use it for comparisons)
        preds.to_csv(PRED_PATH, index=False)
        print(f"Computed supervised actions in {PRED_PATH} using best-F1 threshold {best_t:.3f}")
    else:
        raise RuntimeError("No supervised action column and no full predictions file to compute it.")

print("Using supervised column:", supervised_col)

# 3) Find RL/baseline column (try several names)
rl_candidates = ['rl_action', 'action_03', 'action_05', 'rl_baseline', 'rl_policy']
rl_col = next((c for c in rl_candidates if c in df.columns), None)

# If RL column not present, create a baseline RL action (e.g., threshold 0.3) using full preds file
if rl_col is None:
    if os.path.exists(PRED_PATH):
        preds = pd.read_csv(PRED_PATH)
        if 'pred_prob' not in preds.columns:
            raise RuntimeError("No 'pred_prob' in predictions file to create RL baseline.")
        # conservative baseline: approve if pred_prob < 0.3 (or you used >= depending on encode)
        # In our earlier pipeline supervised_action = (pred_prob < 0.5) meant approve if prob < threshold.
        # Here, we used action=1 meaning approve. Ensure consistent logic.
        # We'll set baseline_approve_if_prob_lt = 0.3 as an example:
        baseline_t = 0.3
        # If earlier supervised_action was defined as (pred_prob < t) -> approve, then:
        # But in our saved preds we used pred_label_0.5 as (pred_prob>=0.5). For clarity, create approve = (pred_prob < baseline_t)
        preds['rl_baseline'] = (preds['pred_prob'] < baseline_t).astype(int)
        preds.to_csv(PRED_PATH, index=False)  # update file
        rl_col = 'rl_baseline'
        print(f"No RL column found. Created RL baseline '{rl_col}' in {PRED_PATH} using threshold {baseline_t}.")
    else:
        raise RuntimeError("No RL column and no predictions file to build a baseline from.")

print("Using RL column:", rl_col)

# 4) Merge if disagreements file missing some columns: align df with preds by index where possible
if supervised_col not in df.columns or rl_col not in df.columns:
    preds = pd.read_csv(PRED_PATH)
    # If the disagreements file was generated as a subset of preds (kept original index), try to merge on index:
    # If there is an 'index' column in df, use it; otherwise attempt left join by values of several key columns.
    if 'index' in df.columns:
        df = df.merge(preds[[supervised_col, rl_col]], left_on='index', right_index=True, how='left')
    else:
        # safest fallback: save a new disagreements file by selecting rows from preds where the policies disagree
        print("Reconstructing disagreements from full predictions using selected columns.")
        if supervised_col not in preds.columns:
            # ensure preds has supervised_col (e.g., action_bestF1)
            if supervised_col == 'action_bestF1' and 'pred_prob' in preds.columns:
                probs = preds['pred_prob'].values
                thresholds = np.linspace(0,1,101)
                f1s = [f1_score(preds['target'].values, (probs>=t).astype(int), zero_division=0) for t in thresholds]
                best_t = thresholds[np.argmax(f1s)]
                preds['action_bestF1'] = (probs >= best_t).astype(int)
        if rl_col not in preds.columns:
            # create rl baseline if not present
            preds[rl_col] = (preds['pred_prob'] < 0.3).astype(int)
        # find disagreements across the whole preds table
        disagree_mask = preds[supervised_col] != preds[rl_col]
        df = preds[disagree_mask].copy()
        df.to_csv(DISAGREE_PATH, index=False)
        print("Reconstructed disagreements CSV at", DISAGREE_PATH)

# 5) Now compute counts and summary
sa = supervised_col
ra = rl_col
counts = df.groupby([sa, ra]).size().unstack(fill_value=0)
print("\nCounts (supervised_action rows vs rl_action cols):")
print(counts)

total = len(df)
sa_approve_rl_deny = counts.loc[1,0] if (1 in counts.index and 0 in counts.columns) else 0
sa_deny_rl_approve = counts.loc[0,1] if (0 in counts.index and 1 in counts.columns) else 0
print(f"\nsupervised approves & RL denies: {sa_approve_rl_deny} ({sa_approve_rl_deny/total:.2%})")
print(f"supervised denies & RL approves: {sa_deny_rl_approve} ({sa_deny_rl_approve/total:.2%})")

# 6) Save cleaned disagreements CSV and print a sample
OUT_PATH = "./analysis/disagreements_sample_cleaned.csv"
df.to_csv(OUT_PATH, index=False)
print("\nSaved cleaned disagreements CSV to:", OUT_PATH)
print("\nSample rows:")
display(df.head(10))


Loaded disagreements file rows: 200
Columns in disagreements CSV:
 ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'revol_bal', 'revol_util', 'total_acc', 'open_acc', 'term_ 36 months', 'term_ 60 months', 'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G', 'sub_grade_A1', 'sub_grade_A2', 'sub_grade_A3', 'sub_grade_A4', 'sub_grade_A5', 'sub_grade_B1', 'sub_grade_B2', 'sub_grade_B3', 'sub_grade_B4', 'sub_grade_B5', 'sub_grade_C1', 'sub_grade_C2', 'sub_grade_C3', 'sub_grade_C4', 'sub_grade_C5', 'sub_grade_D1', 'sub_grade_D2', 'sub_grade_D3', 'sub_grade_D4', 'sub_grade_D5', 'sub_grade_E1', 'sub_grade_E2', 'sub_grade_E3', 'sub_grade_E4', 'sub_grade_E5', 'sub_grade_F1', 'sub_grade_F2', 'sub_grade_F3', 'sub_grade_F4', 'sub_grade_F5', 'sub_grade_G1', 'sub_grade_G2', 'sub_grade_G3', 'sub_grade_G4', 'sub_grade_G5', 'emp_length_1 year', 'emp_length_10+ years', 'emp_length_2 years', 'emp_length_3 years', 'emp_length_4 years', 'emp_length_5 years', 'emp_length

,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,revol_bal,revol_util,total_acc,open_acc,term_ 36 months,...,addr_state_WV,addr_state_WY,loan_amnt_orig,int_rate_orig,target,pred_prob,pred_label_0.5,action_bestF1,action_05,action_03
0,1.715488,1.716741,2.350939,-1.302153,-0.373509,-0.300163,-1.523684,-0.119975,-0.168278,0.0,...,0.0,0.0,30000.0,19.99,0,0.276534,0,1,0,0
1,2.297386,1.022712,1.399247,-0.726972,-0.373509,0.260680,0.025485,0.459086,-0.700630,0.0,...,0.0,0.0,35000.0,16.99,1,0.231997,0,1,0,0
2,2.297386,1.450697,1.249695,0.467016,-0.373509,1.732231,1.042911,-0.368144,0.541524,0.0,...,0.0,0.0,35000.0,18.84,0,0.279096,0,1,0,0
3,1.133590,0.920922,0.028811,0.221330,3.967157,0.403898,0.766573,-0.368144,-0.345729,0.0,...,0.0,0.0,25000.0,16.55,0,0.224840,0,1,0,0
4,0.240376,0.481370,-0.459271,1.642635,-0.373509,0.189962,-0.355527,0.045471,0.364073,0.0,...,0.0,0.0,17325.0,14.65,0,0.247623,0,1,0,0
5,1.715488,1.314204,0.216430,0.633486,-0.373509,0.507788,1.645831,-0.616313,-0.700630,0.0,...,0.0,0.0,30000.0,18.25,0,0.257614,0,1,0,0
6,-0.612104,0.328684,-0.476946,-0.162124,-0.373509,-0.379630,1.197828,-0.947205,-1.055531,0.0,...,0.0,0.0,10000.0,13.99,0,0.201973,0,1,0,0
7,-0.911782,1.156891,-0.232225,-0.390589,0.711658,-0.607337,-0.740726,1.120871,-0.345729,1.0,...,0.0,0.0,7425.0,17.57,1,0.242579,0,1,0,0
8,1.133590,0.328684,0.610702,-0.234452,-0.373509,-0.091776,-0.874708,0.459086,0.718974,0.0,...,0.0,0.0,25000.0,13.99,0,0.199949,0,1,0,0
9,-0.379345,0.407340,-0.164247,1.821734,0.711658,0.477371,1.160146,0.293640,0.718974,0.0,...,0.0,0.0,12000.0,14.33,1,0.229489,0,1,0,0


In [29]:
# Compare average rewards on disagreements
import pandas as pd, numpy as np

df = pd.read_csv("./analysis/disagreements_sample_cleaned.csv")
loan = pd.to_numeric(df.get('loan_amnt_orig', df.get('loan_amnt', 1)), errors='coerce').fillna(0).values
ir = pd.to_numeric(df.get('int_rate_orig', df.get('int_rate', 1)), errors='coerce').fillna(0).values / 100.0
target = df['target'].values if 'target' in df.columns else np.zeros(len(df))

def reward(a, i):
    if a == 0:
        return 0.0
    return loan[i]*ir[i] if target[i]==0 else -loan[i]

sa = df['action_bestF1'].values.astype(int)
ra = df['action_03'].values.astype(int)
r_sa = np.array([reward(a,i) for i,a in enumerate(sa)])
r_ra = np.array([reward(a,i) for i,a in enumerate(ra)])

print("Mean reward (supervised):", r_sa.mean())
print("Mean reward (RL/baseline):", r_ra.mean())
print("Δ Reward (RL - supervised):", r_ra.mean() - r_sa.mean())


Mean reward (supervised): -1404.2685125
Mean reward (RL/baseline): 0.0
Δ Reward (RL - supervised): 1404.2685125
